In [ ]:
import itertools
from pprint import pprint

from sqlalchemy.exc import NoInspectionAvailable
from sqlalchemy.inspection import inspect

from app.models import BaseModel, Submission



In [ ]:
m = inspect(Submission)
m.relationships.keys()

r_form = m.relationships
for n in m.relationships.keys():
    print(n)

In [ ]:
pprint(r_form.__dict__)

# TODO: polymorphic on

In [ ]:
# outputs the inheritance tree
def _r_print_class_tree(root_class, indent=0):
    print(''.join(itertools.repeat(' ', indent)) + str(root_class.__name__))
    
    for child_class in root_class.__subclasses__():
        _r_print_class_tree(child_class, indent=indent + 1)
        
_r_print_class_tree(BaseModel)

In [ ]:
the_graph = {}

def get_vertex_key(cls_orm_model):
    return '{}.{}'.format(cls_orm_model.__module__, cls_orm_model.__name__)

def _r_extract_hierarchy(cls_orm_model, graph:dict):
    # extract the current vertex
    vertex = {
        'key': get_vertex_key(cls_orm_model),
        'name': cls_orm_model.__name__,
        'module': cls_orm_model.__module__,
        'relations': [], 
    }
    graph[vertex['key']] = vertex
    
    # add edges for relationships
    try:
        mapper = inspect(cls_orm_model)
    except NoInspectionAvailable:
        mapper = None
        
    if mapper:
        for relation_name in mapper.relationships.keys():
            relation = mapper.relationships[relation_name]
            vertex['relations'].append({
                'source_name': relation_name,
                'source_columns': [c.name for c in relation.local_columns],
                'is_self_referential': relation._is_self_referential,
                'target': get_vertex_key(relation.argument),
                'taget_columns': [c.name for c in relation.remote_side],
                'back_reference_name': relation.backref,
                'type': relation.direction.name
            })
    
    for child_class in cls_orm_model.__subclasses__():
        vertex['relations'].append({
            'target': get_vertex_key(child_class),
            'type': 'inheritance'
        })
        
        _r_extract_hierarchy(child_class, graph)

    
        
_r_extract_hierarchy(BaseModel, the_graph)
pprint(the_graph)